In [1]:
import pandas as pd

import os
from glob import glob

In [2]:
raiz = '/home/eduardo/repositorios/IS_Agro/databases'


indice_uf = ['elemento', 'fonte', 'uf', 'UF-sigla', 'fluxo']
indice_br = ['elemento', 'fonte', 'fluxo']
indices = (indice_uf, indice_br)

balancos = ('Balanço UF', 'Balanço BR')

arquivos_csv = []
arquivos_csv.extend(glob(os.path.join(raiz, 'ouro_npk_*.csv')))
arquivos_csv.sort()

fontes = (
            'Carcaça Bovina',
            'Dejetos Galináceos',
            'Dejetos Suínos',
            'Deposição Atmosférica',
            'Fertilizante Orgânico vinhaça',
            'Fertilizante Sintético',
            'Fixação Bio N',
            'Outras Produções Agrícolas',
            'Princ. Produções Agrícolas',
            'Sementes'
          )

In [26]:
with pd.ExcelWriter('/home/eduardo/Downloads/balanco_npk.xlsx', mode='w', engine="openpyxl") as export:
    for indice, balanco in zip(indices, balancos):
        df_balanço = (pd.read_csv(f'{raiz}/ouro_npk-balanco.csv')
                      .merge(pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet')
                             .filter(['UF-id', 'UF-sigla']),
                             how='left',
                             left_on='uf',
                             right_on='UF-id'
                             )
                      .pivot_table(
                          index=indice,
                          columns='ano',
                          values='valor'
                          )
                      .sort_values(
                          by=['elemento','fonte'],
                          key=lambda x: x.map({'N': 0, 'P': 1, 'K': 2,
                                            'Fertilizantes Sintéticos':0,
                                            'Fertilizante orgânico vinhaça':1,
                                            'Dejetos galináceos':2,
                                            'Dejetos suínos':3,
                                            'Fixação biológica de N':4,
                                            'Sementes':5,
                                            'Deposição Atmosférica':6,
                                            'Principais produções agrícolas':7,
                                            'Outras produções agrícolas':8,
                                            'Carcaça Bovina':9})
                                    )
                    )
        df_balanço.to_excel(export, sheet_name=balanco, float_format='%.4f')

In [27]:
with pd.ExcelWriter('/home/eduardo/Downloads/balanco_npk.xlsx', mode='a', engine="openpyxl", if_sheet_exists='replace') as export:
    for arquivo, fonte in zip(arquivos_csv, fontes):
        df = (pd.read_csv(arquivo)
              .merge(pd.read_parquet(f'{raiz}/prata_geocodigo_ibge.parquet')
                     .filter(['UF-id', 'UF-sigla']),
                     how='left',
                     left_on='uf',
                     right_on='UF-id'
                     )
              .assign(valor=lambda v: v.valor/1000)
              .pivot_table(
                index=['uf', 'UF-sigla', 'elemento'],
                columns='ano',
                values='valor'
                )
            .reset_index()
            .sort_values(by=['elemento', 'uf'],
                        key=lambda x: x.map({'N': 0, 'P': 1, 'K': 2}))
            .assign(fluxo=lambda x: 'Saída' if arquivo in (arquivos_csv[0], arquivos_csv[7], arquivos_csv[8]) else 'Entrada')
            )
        df.to_excel(export, sheet_name=fonte, float_format='%.4f', index=False)